In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

df = pd.read_csv('data/11.csv', sep = ';')


def get_education(arg):
    arg= arg.lower()
    args_splited = arg.split(' ')
    return args_splited[0]+' '+args_splited[1]
df['Образование'] = df['Образование и ВУЗ'].apply(get_education)

df['Образование'] = df['Образование'].apply(lambda x: 'высшее' if x== 'высшее образование' else x)
df['Образование'] = df['Образование'].apply(lambda x: 'среднее' if x== 'среднее образование' else x)
df = df.drop('Образование и ВУЗ', axis = 1)



def get_age(arg):
    args_splited = arg.split(',')
    a = args_splited[1]
    a_list = a.split()
    age = a_list[0]
    return age
df['Возраст']= df['Пол, возраст'].apply(get_age).astype(int)

df['Пол'] = df['Пол, возраст'].apply(lambda x: 'M' if 'Мужчина' in x else 'Ж')
#удаляем столбец "Пол, возраст"
df = df.drop('Пол, возраст', axis =1)
#выводим ответы на вопросы про процент женщин и средний возраст

df['Опыт работы'] = df['Опыт работы'].astype(str)



def get_exp(arg):
    if arg is np.nan or arg == 'Не указано':
        return None
    month_key_words = ['месяц', 'месяцев', 'месяца']
    year_key_words = ['год', 'лет', 'года']
    month = 0
    year = 0
    args_splited = arg.split()
    args_splited = args_splited[0:7]
    for i in range(len(args_splited)):
        if args_splited[i] in month_key_words:
            month = args_splited[i-1]
        if args_splited[i] in year_key_words:
                year = args_splited[i-1]
    return int(year)*12 + int(month)  
df['Опыт работы (месяц)'] = df['Опыт работы'].apply(get_exp)
df = df.drop('Опыт работы', axis=1)

    
#столбец Город  переезд командировки разбираем
def get_city(arg):
    million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград' ]
    a = ['Москва','Санкт-Петербург']
    city = arg.split()[0]
    if city in million_cities:
        city= 'город-миллионник'
    elif city in a:
        pass
    else:
        city = 'другие'
    return city

df['Город'] = df['Город, переезд, командировки'].apply(get_city)



def get_move(arg):
    a = ['не готов к переезду', 'не готова к переезду'] 
    for i in a:
        if i in arg:
            return False
    return True
        
df['Готовность к переезду'] = df['Город, переезд, командировки'].apply(get_move)

def get_bs_trip(arg):
    a = ['не готов к командировкам', 'не готова к командировкам']
    b = ['готов к командировкам', 'готова к командировкам',
    'готов к редким командировкам', 'готова к редким командировкам'] 
    for i in a:
        if i in arg:
            return False
    for i in b:
        if i in arg:
            return True
    return False
    
df['Готовность к командировкам'] = df['Город, переезд, командировки'].apply(get_bs_trip)
df = df.drop('Город, переезд, командировки', axis = 1)

mask1 = df['Готовность к командировкам'] ==True
mask2 = df['Готовность к переезду'] ==True


employments = ['полная занятость', 'частичная занятость',
              'проектная работа', 'волонтерство', 'стажировка']
charts = ['полный день', 'сменный график', 
         'гибкий график', 'удаленная работа',
         'вахтовый метод']
for employment, chart in zip(employments, charts):
    df[employment] = df['Занятость'].apply(lambda x: employment in x)
    df[chart] = df['График'].apply(lambda x: chart in x)

df = df.drop('Занятость', axis=1)
df = df.drop('График', axis=1)


cur = pd.read_csv('data/ExchangeRates.csv')
cur = cur[['currency', 'date', 'proportion', 'close']]
df['сумма ЗП'] = df['ЗП'].apply(lambda x: int(x.split()[0]))
df['валюта ЗП'] = df['ЗП'].apply(lambda x: x.split()[1])
cur['date']=pd.to_datetime(cur['date'], dayfirst=True).dt.date
df['Обновление резюме']= pd.to_datetime(df['Обновление резюме'], dayfirst=True).dt.date
df['валюта ЗП']= df['валюта ЗП'].apply(lambda x: 'UAH' \
    if (x == 'грн' or x == 'грн.') \
    else 'BYN' if x == 'бел.руб.' \
    else 'UZS' if x == 'сум' \
    else 'РУБ' if x == 'руб.'\
    else x )
    
cur['Обновление резюме'] = cur['date']
cur['валюта ЗП'] = cur['currency']
cur = cur.drop(['currency', 'date'], axis =1)

data = df.merge(cur, on = ['Обновление резюме', 'валюта ЗП'], how = 'left')
values = {'close': 1,'proportion': 1,}
data=data.fillna(values)

data['ЗП(руб)'] = (data['сумма ЗП']*data['close'])/data['proportion']
data=data.drop(['ЗП', 'сумма ЗП', 'валюта ЗП', 'proportion', 'close'], axis=1)

data.to_csv('data/data.csv', index=False, sep=';')




df_do_mln = data[data['ЗП(руб)']<1000000]
fig = px.box(
    data_frame=df_do_mln,
    x="ЗП(руб)", 
    y="Город", 
    color = 'Город',
    height=500, #высота
    width=1000, #ширина
    title='Распределение медианной желаемой ЗП по городам')
fig.show()




NameError: name 'px' is not defined